In [2]:
# !pip install kaggle

In [3]:
import kaggle

In [4]:
!kaggle competitions download -c spaceship-titanic


  0%|          | 0.00/299k [00:00<?, ?B/s]
100%|##########| 299k/299k [00:00<00:00, 412kB/s]
100%|##########| 299k/299k [00:00<00:00, 411kB/s]
